In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import feature_process_helper
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
%matplotlib inline

# RF Model 1

#### Load data

In [2]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))

#### Transform variables

In [3]:
train, test = feature_process_helper.names(train, test)
train, test = feature_process_helper.age_impute(train, test)
train, test = feature_process_helper.cabin(train, test)
train, test = feature_process_helper.embarked_impute(train, test)
train, test = feature_process_helper.dummies(train, test)
train, test = feature_process_helper.drop(train, test)

#### Tune hyper-parameters

In [4]:
rf1 = RandomForestClassifier(max_features='auto',
                                oob_score=True,
                                random_state=1,
                                n_jobs=-1)

In [6]:
param_grid = { "criterion"   : ["gini", "entropy"],
             "min_samples_leaf" : [1,5,10],
             "min_samples_split" : [2, 4, 10, 12, 16],
             "n_estimators": [50, 100, 400, 700]}

gs = GridSearchCV(estimator=rf1,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=3,
                  n_jobs=-1)

gs = gs.fit(train.iloc[:, 2:], train.iloc[:, 1])

In [7]:
print(gs.best_score_)
print(gs.best_params_)
#print(gs.cv_results_)

0.826038159371
{'min_samples_split': 10, 'n_estimators': 100, 'criterion': 'gini', 'min_samples_leaf': 1}


#### Fit model

In [9]:
rf1 = RandomForestClassifier(criterion='gini', 
                             n_estimators=100,
                             min_samples_split=10,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf1.fit(train.iloc[:, 2:], train.iloc[:, 1])
print "%.4f" % rf1.oob_score_ 

0.8238


#### Obtain cross-validation score with optimal hyperparameters

In [16]:
scores1 = cross_val_score(rf1, train.iloc[:, 2:], train.iloc[:, 1], n_jobs=-1)
scores1.mean()

0.82603815937149283

#### Inspect feature ranking

In [21]:
pd.concat((pd.DataFrame(train.iloc[:, 2:].columns, columns = ['variable']), 
           pd.DataFrame(rf1.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

,variable,importance
1,Fare,0.142931
20,Name_Title_Mr.,0.131256
7,Sex_male,0.111903
2,Name_Len,0.107163
0,Age,0.098652
8,Sex_female,0.090083
4,Pclass_3,0.063281
21,Name_Title_Mrs.,0.037778
22,Name_Title_Miss.,0.035874
12,Cabin_Letter_n,0.034054


#### Generate submission file

In [ ]:
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
predictions = rf1.predict(test.iloc[:, 1:])
predictions = pd.DataFrame(predictions, columns=['Survived'])
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv(os.path.join('submission_files', 'y_test1.csv'), sep=",", index = False)

# RF Model 2

#### Load data

In [4]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))

#### Transform variables

In [5]:
train, test = feature_process_helper.names(train, test)
train, test = feature_process_helper.age_impute(train, test)
train, test = feature_process_helper.cabin(train, test)
train, test = feature_process_helper.embarked_impute(train, test)
train, test = feature_process_helper.dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked', 'Cabin_Letter', 'Name_Title',
                                                                    'SibSp', 'Parch'])
train, test = feature_process_helper.drop(train, test, bye = ['Ticket'])

#### Tune hyperparameters

In [6]:
rf1 = RandomForestClassifier(max_features='auto',
                                oob_score=True,
                                random_state=1,
                                n_jobs=-1)

In [7]:
param_grid = { "criterion"   : ["gini", "entropy"],
             "min_samples_leaf" : [1,5,10],
             "min_samples_split" : [2, 4, 10, 12, 16],
             "n_estimators": [50, 100, 400, 700]}

gs = GridSearchCV(estimator=rf1,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=3,
                  n_jobs=-1)

gs = gs.fit(train.iloc[:, 2:], train.iloc[:, 1])

In [8]:
print(gs.best_score_)
print(gs.best_params_)
#print(gs.cv_results_)

0.824915824916
{'min_samples_split': 12, 'n_estimators': 50, 'criterion': 'gini', 'min_samples_leaf': 1}


#### Fit model

In [9]:
rf2 = RandomForestClassifier(criterion='gini', 
                             n_estimators=50,
                             min_samples_split=12,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf2.fit(train.iloc[:, 2:], train.iloc[:, 1])
print "%.4f" % rf2.oob_score_ 

0.8238


#### Obtain cross-validation score with optimal hyperparameters

In [10]:
scores2 = cross_val_score(rf1, train.iloc[:, 2:], train.iloc[:, 1], n_jobs=-1)
scores2.mean()

0.80359147025813693

#### Inspect feature ranking

In [12]:
pd.concat((pd.DataFrame(train.iloc[:, 2:].columns, columns = ['variable']), 
           pd.DataFrame(rf2.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

,variable,importance
20,Name_Title_Mr.,0.116731
7,Sex_male,0.115737
8,Sex_female,0.112738
2,Name_Len,0.105312
1,Fare,0.103418
0,Age,0.084431
4,Pclass_3,0.062422
22,Name_Title_Miss.,0.035632
21,Name_Title_Mrs.,0.032156
5,Pclass_1,0.024577


#### Generate submission file

In [13]:
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
predictions = rf2.predict(test.iloc[:, 1:])
predictions = pd.DataFrame(predictions, columns=['Survived'])
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv(os.path.join('submission_files', 'y_test1.csv'), sep=",", index = False)

In [15]:
train.columns

Index([u'PassengerId', u'Survived', u'Age', u'Fare', u'Name_Len',
       u'Age_Null_Flag', u'Pclass_3', u'Pclass_1', u'Pclass_2', u'Sex_male',
       u'Sex_female', u'Embarked_S', u'Embarked_C', u'Embarked_Q',
       u'Cabin_Letter_n', u'Cabin_Letter_C', u'Cabin_Letter_E',
       u'Cabin_Letter_G', u'Cabin_Letter_D', u'Cabin_Letter_A',
       u'Cabin_Letter_B', u'Cabin_Letter_F', u'Name_Title_Mr.',
       u'Name_Title_Mrs.', u'Name_Title_Miss.', u'Name_Title_Master.',
       u'Name_Title_Rev.', u'Name_Title_Dr.', u'Name_Title_Ms.',
       u'Name_Title_Col.', u'SibSp_1', u'SibSp_0', u'SibSp_3', u'SibSp_4',
       u'SibSp_2', u'SibSp_5', u'SibSp_8', u'Parch_0', u'Parch_1', u'Parch_2',
       u'Parch_5', u'Parch_3', u'Parch_4', u'Parch_6'],
      dtype='object')

# Model 3

In [10]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))

In [11]:
train, test = feature_process_helper.names(train, test)
train, test = feature_process_helper.age_impute(train, test)
train, test = feature_process_helper.cabin(train, test)
train, test = feature_process_helper.embarked_impute(train, test)
train, test = feature_process_helper.dummies(train, test)
train, test = feature_process_helper.drop(train, test)

In [12]:
y_train = train.iloc[:, 1]

In [13]:
del train['Survived']

In [15]:
rf3 = RandomForestClassifier(max_features='auto',
                                oob_score=True,
                                random_state=1,
                                n_jobs=-1)

In [16]:
param_grid = { "criterion"   : ["gini", "entropy"],
             "min_samples_leaf" : [1,5,10],
             "min_samples_split" : [2, 4, 10, 12, 16],
             "n_estimators": [50, 100, 400, 700]}

gs = GridSearchCV(estimator=rf3,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=3,
                  n_jobs=-1)

gs = gs.fit(train, y_train)

In [17]:
print(gs.best_score_)
print(gs.best_params_)
#print(gs.cv_results_)

0.83164983165
{'min_samples_split': 10, 'n_estimators': 400, 'criterion': 'entropy', 'min_samples_leaf': 1}


In [20]:
rf3 = RandomForestClassifier(min_samples_split=10, 
                             n_estimators=400, 
                             criterion='entropy', 
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf3.fit(train, y_train)
print "%.4f" % rf3.oob_score_ 

0.8249


In [21]:
scores3 = cross_val_score(rf3, train, y_train, n_jobs=-1)
scores3.mean()

0.83164983164983164

In [22]:
pd.concat((pd.DataFrame(train.iloc[:, 2:].columns, columns = ['variable']), 
           pd.DataFrame(rf3.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

,variable,importance
2,Age_Null_Flag,0.133448
3,Pclass_3,0.107066
0,Fare,0.103650
1,Name_Len,0.103143
21,Name_Title_Miss.,0.099552
9,Embarked_C,0.092503
8,Embarked_S,0.087784
5,Pclass_2,0.052513
13,Cabin_Letter_E,0.033119
6,Sex_male,0.026176


In [32]:
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
predictions = rf3.predict(test)
predictions = pd.DataFrame(predictions, columns=['Survived'])
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv(os.path.join('submission_files', 'y_test3.csv'), sep=",", index = False)

# RF Model 3

In [4]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))
train, test = feature_process_helper.names(train, test)
train, test = feature_process_helper.age_impute(train, test)
train, test = feature_process_helper.cabin(train, test)
train, test = feature_process_helper.embarked_impute(train, test)
train, test = feature_process_helper.fam_size(train, test)
train, test = feature_process_helper.dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked',
                                                                     'Cabin_Letter', 'Name_Title', 'Fam_Size'])
train, test = feature_process_helper.drop(train, test)

In [5]:
rf1 = RandomForestClassifier(max_features='auto',
                                oob_score=True,
                                random_state=1,
                                n_jobs=-1)

param_grid = { "criterion"   : ["gini", "entropy"],
             "min_samples_leaf" : [1,5,10],
             "min_samples_split" : [2, 4, 10, 12, 16],
             "n_estimators": [50, 100, 400, 700]}

gs = GridSearchCV(estimator=rf1,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=3,
                  n_jobs=-1)

gs = gs.fit(train.iloc[:, 2:], train.iloc[:, 1])

In [6]:
print(gs.best_score_)
print(gs.best_params_)
#print(gs.cv_results_)

0.827160493827
{'min_samples_split': 12, 'n_estimators': 50, 'criterion': 'gini', 'min_samples_leaf': 5}


In [7]:
rf3 = RandomForestClassifier(criterion='gini', 
                             n_estimators=50,
                             min_samples_split=12,
                             min_samples_leaf=5,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf3.fit(train.iloc[:, 2:], train.iloc[:, 1])
print "%.4f" % rf3.oob_score_ 

0.8406


In [8]:
scores3 = cross_val_score(rf3, train.iloc[:, 2:], train.iloc[:, 1], n_jobs=-1)
scores3.mean()

0.8271604938271605

In [9]:
pd.concat((pd.DataFrame(train.iloc[:, 2:].columns, columns = ['variable']), 
           pd.DataFrame(rf3.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

,variable,importance
7,Sex_male,0.190479
20,Name_Title_Mr.,0.147180
1,Fare,0.099675
8,Sex_female,0.091025
4,Pclass_3,0.071663
0,Age,0.071243
2,Name_Len,0.069634
12,Cabin_Letter_n,0.043399
30,Fam_Size_Big,0.034171
21,Name_Title_Mrs.,0.030305


In [10]:
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
predictions = rf3.predict(test.iloc[:, 1:])
predictions = pd.DataFrame(predictions, columns=['Survived'])
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv(os.path.join('submission_files', 'y_test3.csv'), sep=",", index = False)

Leaderboard score: 0.80383  
1024 out of 6067 (top 17%)

# Model 4


In [26]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
train, test = feature_process_helper.names(train, test)
train, test = feature_process_helper.age_impute(train, test)
train, test = feature_process_helper.cabin(train, test)
train, test = feature_process_helper.embarked_impute(train, test)
train, test = feature_process_helper.fam_size(train, test)
train, test = feature_process_helper.dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked',
                                                                     'Cabin_Letter', 'Name_Title', 'Fam_Size'])
train, test = feature_process_helper.drop(train, test)
train, test = feature_process_helper.lda(train, test, train.iloc[:, 1], cols=['Age', 'Fare'])

In [27]:
rf4 = RandomForestClassifier(max_features='auto',
                                oob_score=True,
                                random_state=1,
                                n_jobs=-1)

param_grid = { "criterion"   : ["gini", "entropy"],
             "min_samples_leaf" : [1,5,10],
             "min_samples_split" : [2, 4, 10, 12, 16],
             "n_estimators": [50, 100, 400, 700]}

gs = GridSearchCV(estimator=rf4,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=3,
                  n_jobs=-1)

gs = gs.fit(train.iloc[:, 2:], train.iloc[:, 1])

In [28]:
print(gs.best_score_)
print(gs.best_params_)
#print(gs.cv_results_)

0.828282828283
{'min_samples_split': 10, 'n_estimators': 50, 'criterion': 'entropy', 'min_samples_leaf': 1}


In [29]:
rf4 = RandomForestClassifier(criterion='entropy', 
                             n_estimators=50,
                             min_samples_split=10,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf4.fit(train.iloc[:, 2:], train.iloc[:, 1])
print "%.4f" % rf4.oob_score_

0.8126


In [30]:
scores4 = cross_val_score(rf4, train.iloc[:, 2:], train.iloc[:, 1], n_jobs=-1)
scores4.mean()

0.82828282828282829

In [31]:
pd.concat((pd.DataFrame(train.iloc[:, 2:].columns, columns = ['variable']), 
           pd.DataFrame(rf4.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

,variable,importance
29,0,0.194866
0,Name_Len,0.133324
6,Sex_female,0.116406
18,Name_Title_Mr.,0.094739
5,Sex_male,0.082101
2,Pclass_3,0.064193
10,Cabin_Letter_n,0.035912
28,Fam_Size_Big,0.033476
19,Name_Title_Mrs.,0.030457
3,Pclass_1,0.024086


In [32]:
predictions = rf4.predict(test.iloc[:, 1:])
predictions = pd.DataFrame(predictions, columns=['Survived'])
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv(os.path.join('submission_files', 'y_test4.csv'), sep=",", index = False)

Leaderboard score: 0.78

# Model 5

In [2]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
train, test = feature_process_helper.names(train, test)
train, test = feature_process_helper.age_impute(train, test)
train, test = feature_process_helper.cabin(train, test)
train, test = feature_process_helper.embarked_impute(train, test)
train, test = feature_process_helper.fam_size(train, test)
train, test = feature_process_helper.titles_grouped(train, test)
train, test = feature_process_helper.dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked',
                                                                     'Cabin_Letter', 'Name_Title', 'Fam_Size'])
train, test = feature_process_helper.drop(train, test)

In [3]:
rf5 = RandomForestClassifier(max_features='auto',
                                oob_score=True,
                                random_state=1,
                                n_jobs=-1)

param_grid = { "criterion"   : ["gini", "entropy"],
             "min_samples_leaf" : [1,5,10],
             "min_samples_split" : [2, 4, 10, 12, 16],
             "n_estimators": [50, 100, 400, 700]}

gs = GridSearchCV(estimator=rf5,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=3,
                  n_jobs=-1)

gs = gs.fit(train.iloc[:, 2:], train.iloc[:, 1])

In [4]:
print(gs.best_score_)
print(gs.best_params_)
#print(gs.cv_results_)

0.829405162738
{'min_samples_split': 12, 'n_estimators': 50, 'criterion': 'entropy', 'min_samples_leaf': 1}


In [5]:
rf5 = RandomForestClassifier(criterion='entropy', 
                             n_estimators=50,
                             min_samples_split=12,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf5.fit(train.iloc[:, 2:], train.iloc[:, 1])
print "%.4f" % rf5.oob_score_

0.8260


In [6]:
scores5 = cross_val_score(rf5, train.iloc[:, 2:], train.iloc[:, 1], n_jobs=-1)
scores5.mean()

0.82940516273849607

In [7]:
pd.concat((pd.DataFrame(train.iloc[:, 2:].columns, columns = ['variable']), 
           pd.DataFrame(rf5.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

,variable,importance
1,Fare,0.124604
0,Age,0.121088
2,Name_Len,0.106165
20,Name_Title_Mr.,0.099116
7,Sex_male,0.096142
8,Sex_female,0.083227
4,Pclass_3,0.051289
21,Name_Title_Mrs.,0.035439
22,Name_Title_Miss.,0.031467
5,Pclass_1,0.029258


In [8]:
predictions = rf5.predict(test.iloc[:, 1:])
predictions = pd.DataFrame(predictions, columns=['Survived'])
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv(os.path.join('submission_files', 'y_test5.csv'), sep=",", index = False)

Leaderboard score: 0.79904

# RF 6

In [9]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))
train, test = feature_process_helper.names(train, test)
train, test = feature_process_helper.age_impute(train, test)
train, test = feature_process_helper.cabin(train, test)
train, test = feature_process_helper.embarked_impute(train, test)
train, test = feature_process_helper.fam_size(train, test)
train, test = feature_process_helper.dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked',
                                                                     'Cabin_Letter', 'Name_Title', 'Fam_Size'])
train, test = feature_process_helper.drop(train, test)

In [10]:
rf3 = RandomForestClassifier(criterion='gini', 
                             n_estimators=300,
                             min_samples_split=12,
                             min_samples_leaf=5,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf3.fit(train.iloc[:, 2:], train.iloc[:, 1])
print "%.4f" % rf3.oob_score_ 

0.8305


In [11]:
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
predictions = rf3.predict(test.iloc[:, 1:])
predictions = pd.DataFrame(predictions, columns=['Survived'])
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv(os.path.join('submission_files', 'y_test6.csv'), sep=",", index = False)

Leaderboard score: 0.80861  
563 out of 6067 (top 9.3%)

# RF 7

In [3]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))
train, test = feature_process_helper.names(train, test)
train, test = feature_process_helper.age_impute(train, test)
train, test = feature_process_helper.cabin(train, test)
train, test = feature_process_helper.embarked_impute(train, test)
train, test = feature_process_helper.fam_size(train, test)
train, test = feature_process_helper.ticket_grouped(train, test)
train, test = feature_process_helper.dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked', 'Ticket_Lett',
                                                                     'Cabin_Letter', 'Name_Title', 'Fam_Size'])
train, test = feature_process_helper.drop(train, test)

In [5]:
rf7 = RandomForestClassifier(max_features='auto',
                                oob_score=True,
                                random_state=1,
                                n_jobs=-1)

param_grid = { "criterion"   : ["gini", "entropy"],
             "min_samples_leaf" : [1,5,10],
             "min_samples_split" : [2, 4, 10, 12, 16],
             "n_estimators": [50, 100, 400, 700]}

gs = GridSearchCV(estimator=rf7,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=3,
                  n_jobs=-1)

gs = gs.fit(train.iloc[:, 2:], train.iloc[:, 1])

In [6]:
print(gs.best_score_)
print(gs.best_params_)
#print(gs.cv_results_)

0.838383838384
{'min_samples_split': 4, 'n_estimators': 100, 'criterion': 'gini', 'min_samples_leaf': 1}


In [7]:
rf7 = RandomForestClassifier(criterion='gini', 
                             n_estimators=100,
                             min_samples_split=4,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf7.fit(train.iloc[:, 2:], train.iloc[:, 1])
print "%.4f" % rf7.oob_score_

0.8283


In [8]:
scores7 = cross_val_score(rf7, train.iloc[:, 2:], train.iloc[:, 1], n_jobs=-1)
scores7.mean()

0.83838383838383834

In [9]:
pd.concat((pd.DataFrame(train.iloc[:, 2:].columns, columns = ['variable']), 
           pd.DataFrame(rf7.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

,variable,importance
1,Fare,0.120061
2,Name_Len,0.117291
0,Age,0.116984
29,Name_Title_Mr.,0.111207
8,Sex_female,0.082661
7,Sex_male,0.074994
4,Pclass_3,0.039930
39,Fam_Size_Big,0.028404
21,Cabin_Letter_n,0.028148
31,Name_Title_Miss.,0.027901


In [10]:
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
predictions = rf7.predict(test.iloc[:, 1:])
predictions = pd.DataFrame(predictions, columns=['Survived'])
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv(os.path.join('submission_files', 'y_test7.csv'), sep=",", index = False)

Leaderboard score: 0.7894

# RF 8

I tried RF 7 with 500 estimators, and the score went down to .7799

# RF 9

In [24]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))
train, test = feature_process_helper.names(train, test)
train, test = feature_process_helper.age_impute(train, test)
train, test = feature_process_helper.cabin(train, test)
train, test = feature_process_helper.embarked_impute(train, test)
train, test = feature_process_helper.fam_size(train, test)
train, test = feature_process_helper.dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked', 
                                                                     'Cabin_Letter', 'Name_Title', 'Fam_Size'])
train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x))
test['Ticket_Len'] = test['Ticket'].apply(lambda x: len(x))
train, test = feature_process_helper.drop(train, test)

In [16]:
rf9 = RandomForestClassifier(max_features='auto',
                                oob_score=True,
                                random_state=1,
                                n_jobs=-1)

param_grid = { "criterion"   : ["gini", "entropy"],
             "min_samples_leaf" : [1,5,10],
             "min_samples_split" : [2, 4, 10, 12, 16],
             "n_estimators": [50, 100, 400, 700]}

gs = GridSearchCV(estimator=rf9,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=3,
                  n_jobs=-1)

gs = gs.fit(train.iloc[:, 2:], train.iloc[:, 1])

In [17]:
print(gs.best_score_)
print(gs.best_params_)
#print(gs.cv_results_)

0.836139169473
{'min_samples_split': 4, 'n_estimators': 50, 'criterion': 'entropy', 'min_samples_leaf': 1}


In [18]:
rf9 = RandomForestClassifier(criterion='entropy', 
                             n_estimators=50,
                             min_samples_split=4,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf9.fit(train.iloc[:, 2:], train.iloc[:, 1])
print "%.4f" % rf9.oob_score_

0.8227


In [19]:
scores9 = cross_val_score(rf9, train.iloc[:, 2:], train.iloc[:, 1], n_jobs=-1)
scores9.mean()

0.83613916947250277

In [20]:
pd.concat((pd.DataFrame(train.iloc[:, 2:].columns, columns = ['variable']), 
           pd.DataFrame(rf9.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

,variable,importance
2,Name_Len,0.136529
1,Fare,0.135527
0,Age,0.132001
7,Sex_male,0.083224
8,Sex_female,0.073391
20,Name_Title_Mr.,0.070640
31,Ticket_Len,0.063579
4,Pclass_3,0.040833
12,Cabin_Letter_n,0.026080
30,Fam_Size_Big,0.025057


In [26]:
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
predictions = rf9.predict(test.iloc[:, 1:])
predictions = pd.DataFrame(predictions, columns=['Survived'])
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv(os.path.join('submission_files', 'y_test9.csv'), sep=",", index = False)

Leaderboard score: 0.78469

# Model 10

In [27]:
rf10 = RandomForestClassifier(criterion='gini', 
                             n_estimators=300,
                             min_samples_split=12,
                             min_samples_leaf=5,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf10.fit(train.iloc[:, 2:], train.iloc[:, 1])
print "%.4f" % rf10.oob_score_ 

0.8316


In [28]:
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
predictions = rf10.predict(test.iloc[:, 1:])
predictions = pd.DataFrame(predictions, columns=['Survived'])
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv(os.path.join('submission_files', 'y_test10.csv'), sep=",", index = False)

Leaderborad score: 0.81340  
445 out of 6069 (top 8%)

# Model 11

In [29]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))
train, test = feature_process_helper.names(train, test)
train, test = feature_process_helper.age_impute(train, test)
train, test = feature_process_helper.cabin(train, test)
train, test = feature_process_helper.embarked_impute(train, test)
train, test = feature_process_helper.fam_size(train, test)
train, test = feature_process_helper.ticket_grouped(train, test)
train, test = feature_process_helper.dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked', 'Ticket_Lett',
                                                                     'Cabin_Letter', 'Name_Title', 'Fam_Size'])
train, test = feature_process_helper.drop(train, test)

In [30]:
rf11 = RandomForestClassifier(criterion='gini', 
                             n_estimators=300,
                             min_samples_split=12,
                             min_samples_leaf=5,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf11.fit(train.iloc[:, 2:], train.iloc[:, 1])
print "%.4f" % rf11.oob_score_ 

0.8283


In [31]:
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
predictions = rf11.predict(test.iloc[:, 1:])
predictions = pd.DataFrame(predictions, columns=['Survived'])
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv(os.path.join('submission_files', 'y_test11.csv'), sep=",", index = False)

Leaderboard score: 0.81340  
445 out of 6069 (top 8%)

# Model 12

In [32]:
rf12 = RandomForestClassifier(criterion='gini', 
                             n_estimators=1000,
                             min_samples_split=12,
                             min_samples_leaf=5,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf12.fit(train.iloc[:, 2:], train.iloc[:, 1])
print "%.4f" % rf12.oob_score_ 

0.8350


In [33]:
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
predictions = rf12.predict(test.iloc[:, 1:])
predictions = pd.DataFrame(predictions, columns=['Survived'])
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv(os.path.join('submission_files', 'y_test12.csv'), sep=",", index = False)

Leaderboard score: 